In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
train.shape

(19579, 3)

In [4]:
train[:5]

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

dataset = train.append(test)
dataset.shape
word_dict = CountVectorizer().fit(dataset["text"])
word_dict.vocabulary_

{'inexpressible': 12903,
 'mahmoud': 15025,
 'translator': 25496,
 'at': 1620,
 'tumbling': 25767,
 'contents': 5189,
 'oslo': 17287,
 'irregularity': 13535,
 'singer': 22655,
 'mourners': 16139,
 'gorge': 10939,
 'outspeeded': 17386,
 'bulk': 3229,
 'alarmed': 645,
 'coaches': 4458,
 'bondage': 2819,
 'macabre': 14952,
 'lungs': 14893,
 'horthath': 12073,
 'economics': 7884,
 'prevailed': 19187,
 'accorded': 181,
 'meddling': 15433,
 'items': 13607,
 'disquisition': 7190,
 'literally': 14613,
 'past': 17841,
 'effect': 7944,
 'northerly': 16724,
 'standards': 23515,
 'dripping': 7601,
 'expectations': 9020,
 'report': 20697,
 'confirmation': 4953,
 'cameleopards': 3454,
 'infallible': 12910,
 'swelters': 24390,
 'scrap': 21870,
 'debris': 6138,
 'reheaded': 20461,
 'yxu': 28227,
 'crazed': 5605,
 'wearin': 27544,
 'peeled': 17983,
 'unnaturally': 26407,
 'uncomfortably': 26029,
 'credential': 5632,
 'preposterous': 19119,
 'break': 3002,
 'ruler': 21405,
 'disgusted': 7080,
 'building

In [6]:
import numpy as np

np.array(list(word_dict.vocabulary_.values())).min()

0

In [7]:
# import nltk
# nltk.download('punkt')

In [7]:
from nltk import word_tokenize

invalid_word_idx = -1

train["encoded"] = train["text"].apply(lambda x: 
                              [word_dict.vocabulary_[word.lower()] 
                               if word.lower() in word_dict.vocabulary_ 
                               else invalid_word_idx
                               for word in word_tokenize(x) 
                               if word.isalpha() ])
train["encoded_len"] = train["encoded"].apply(lambda x: len(x))
train[:5]                              

,id,text,author,encoded,encoded_len
0,id26305,"This process, however, afforded me no means of...",EAP,"[24989, 19300, 12122, 510, 15383, 16655, 15403...",41
1,id17569,It never once occurred to me that the fumbling...,HPL,"[13597, 16557, 17117, 16984, 25229, 15383, 248...",14
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[12623, 11895, 14354, 11396, 27455, -1, 10914,...",36
3,id27763,How lovely is spring As we looked from Windsor...,MWS,"[12121, 14825, 13566, 23392, 1456, 27520, 1475...",34
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[9667, 16753, 8113, 16744, 8784, 10914, 24886,...",27


In [9]:
train["encoded_len"].describe()

count    19579.000000
mean        26.639512
std         19.003268
min          0.000000
25%         15.000000
50%         23.000000
75%         34.000000
max        862.000000
Name: encoded_len, dtype: float64

Явно има изречения с по 0 думи, ще ги филтрирам не виждам как ще носят информация :)

In [8]:
train = train [train["encoded_len"] > 2]
train["encoded_len"].describe()

count    19572.000000
mean        26.648426
std         19.000813
min          3.000000
25%         15.000000
50%         23.000000
75%         34.000000
max        862.000000
Name: encoded_len, dtype: float64

In [9]:
max_sentence_len = int(train["encoded_len"].describe()["max"])
max_sentence_len

862

Ще подравня всички изречения да имат равен брой думи, за да мога да ползвам фиксиран ембединг

In [10]:
train_padded = pd.DataFrame()
def padd_sentence(sentence):
    diff = max_sentence_len - len(sentence)
    for _ in range(diff):
        sentence.append(invalid_word_idx)
    
    return sentence
    
train_padded["encoded"] = train["encoded"].apply(padd_sentence)
train_padded["encoded_len"] = train_padded["encoded"].apply(lambda x: len(x))
train_padded[:5]

,encoded,encoded_len
0,"[24989, 19300, 12122, 510, 15383, 16655, 15403...",862
1,"[13597, 16557, 17117, 16984, 25229, 15383, 248...",862
2,"[12623, 11895, 14354, 11396, 27455, -1, 10914,...",862
3,"[12121, 14825, 13566, 23392, 1456, 27520, 1475...",862
4,"[9667, 16753, 8113, 16744, 8784, 10914, 24886,...",862


In [11]:
x_train = np.array([ sentence for sentence in train_padded["encoded"]])
x_train.shape

(19572, 862)

In [12]:
train["author"] = train["author"].replace("EAP",0)
train["author"] = train["author"].replace("HPL",1)
train["author"] = train["author"].replace("MWS",2)

from sklearn.preprocessing import OneHotEncoder

y_train = OneHotEncoder().fit_transform(train["author"].as_matrix().reshape(-1, 1))
y_train.shape

(19572, 3)

In [13]:
import keras
from keras.layers import Dense, Embedding, Bidirectional, Input, LSTM, Bidirectional
from keras.models import Model

Using TensorFlow backend.


In [14]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8676499918070679944
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2534080512
locality {
  bus_id: 1
}
incarnation: 7368396669237492378
physical_device_desc: "device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


In [77]:
sequence_input = Input(shape=(max_sentence_len,))
embedding_layer = Embedding(input_dim=max_sentence_len, output_dim=128 )
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(128))(embedded_sequences)
preds = Dense(3, activation="softmax")(l_lstm)
model = Model(sequence_input, preds)

In [78]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["acc"])


In [79]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 862)               0         
_________________________________________________________________
embedding_27 (Embedding)     (None, 862, 128)          110336    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 771       
Total params: 374,275
Trainable params: 374,275
Non-trainable params: 0
_________________________________________________________________


In [21]:
y_train = y_train.todense()

In [ ]:
model.fit(x_train, y_train,epochs=16, validation_split=0.2, batch_size=256)

In [ ]:
model.save("bidirection_lstm.h5") # loss > 1 

Ще пробвам нов модел с по-малък embedding layer и 1 lstm

In [ ]:
sequence_input = Input(shape=(max_sentence_len,))
embedding_layer = Embedding(input_dim=max_sentence_len, output_dim=64 )
embedded_sequences = embedding_layer(sequence_input)
lstm = LSTM(64)(embedded_sequences)
preds = Dense(3, activation="softmax")(lstm)
model = Model(sequence_input, preds)

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["acc"])


In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train,epochs=12, validation_split=0.2, batch_size=256)

In [ ]:
model.save("lstm.h5") # loss > 1

Не постигнах кой знае какъв успех, ще пробвам още веднъж с различен оптимизатор и регуларизация

In [16]:
from keras.models import Sequential
model = Sequential([
    Embedding(input_dim=max_sentence_len, output_dim=128),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation="softmax")
])

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])


In [128]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_44 (Embedding)     (None, 862, 128)          110336    
_________________________________________________________________
dropout_28 (Dropout)         (None, 862, 128)          0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 110336)            0         
_________________________________________________________________
dense_61 (Dense)             (None, 226)               24936162  
_________________________________________________________________
dropout_29 (Dropout)         (None, 226)               0         
_________________________________________________________________
dense_62 (Dense)             (None, 3)                 681       
Total params: 25,047,179
Trainable params: 25,047,179
Non-trainable params: 0
________________________________________________________________

In [ ]:
model.fit(x_train, y_train, batch_size=512,epochs=15,validation_split=0.2)

In [ ]:
# wont save it loss > 1

Ще пробвам и с нормална feed forward, но с embedding

In [23]:
from keras.layers.recurrent import SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dropout, Reshape, Flatten

model = Sequential([
    Embedding(max_sentence_len, 84, input_length=max_sentence_len),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation="tanh"),
    Dropout(0.2),
    Dense(32, activation="tanh"),
    Dense(16, activation="tanh"),
    Dense(3, activation="softmax")
])

In [24]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])


In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 862, 84)           72408     
_________________________________________________________________
dropout_3 (Dropout)          (None, 862, 84)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 72408)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4634176   
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 16)                528       
__________

In [26]:
model.fit(x_train, y_train, batch_size=256,epochs=12,validation_split=0.2)

Train on 15657 samples, validate on 3915 samples
Epoch 1/12
15657/15657 [==============================] - 2s 140us/step - loss: 1.0933 - acc: 0.4049 - val_loss: 1.0905 - val_acc: 0.4061
Epoch 2/12
15657/15657 [==============================] - 2s 123us/step - loss: 1.0799 - acc: 0.4139 - val_loss: 1.0937 - val_acc: 0.4059
Epoch 3/12
15657/15657 [==============================] - 2s 123us/step - loss: 1.0535 - acc: 0.4404 - val_loss: 1.0869 - val_acc: 0.4314
Epoch 4/12
15657/15657 [==============================] - 2s 123us/step - loss: 1.0222 - acc: 0.4836 - val_loss: 1.1266 - val_acc: 0.4268
Epoch 5/12
15657/15657 [==============================] - 2s 122us/step - loss: 1.0266 - acc: 0.4865 - val_loss: 1.1070 - val_acc: 0.4207
Epoch 6/12
15657/15657 [==============================] - 2s 124us/step - loss: 1.0245 - acc: 0.4821 - val_loss: 1.0973 - val_acc: 0.4204
Epoch 7/12
15657/15657 [==============================] - 2s 124us/step - loss: 1.0241 - acc: 0.4808 - val_loss: 1.1053 - v